In [10]:
# --- Day 11: Single source of truth for league radar ---

from pathlib import Path
import sys

# Anchor to repo root (.../nba)
PROJECT_ROOT = Path.cwd().parents[0]
sys.path.append(str(PROJECT_ROOT))

from src.data_loader import load_default_2024_25
from src.radar import build_league_radar

# Load canonical dataset
df = load_default_2024_25()

# Build league-wide replacement radar
radar = build_league_radar(
    df,
    star_min_threshold=30.0,
    min_games_without=5
)

radar.shape


(1309, 11)

In [11]:
from src.data_loader import load_default_2024_25


In [12]:
df = load_default_2024_25()
df.shape


(37727, 37)

In [13]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().parents[0]  # .../nba (since notebook lives in .../nba/notebooks)
sys.path.append(str(PROJECT_ROOT))

from src.data_loader import load_default_2024_25
from src.radar import build_league_radar

df = load_default_2024_25()
radar = build_league_radar(df, star_min_threshold=30.0, min_games_without=5)

radar.shape, radar.head(5)


((1309, 11),
                      STAR_OUT      TEAM_CITY TEAM_NAME       BENEFICIARY  \
 1192               Trae Young        Atlanta     Hawks    Keaton Wallace   
 1050              Paul George   Philadelphia     76ers       Chuma Okeke   
 1209          Trey Murphy III    New Orleans  Pelicans  Keion Brooks Jr.   
 1158  Shai Gilgeous-Alexander  Oklahoma City   Thunder      Dillon Jones   
 657               Joel Embiid   Philadelphia     76ers       Chuma Okeke   
 
           SCORE  DELTA_PTS  DELTA_AST  DELTA_REB  DELTA_MIN  N_WITHOUT  \
 1192  38.849074  10.344444   5.977778   3.522222  19.004630          6   
 1050  38.500000   8.800000   2.200000   7.000000  20.500000          5   
 1209  35.504747   9.972222   0.805556   3.416667  21.310303         12   
 1158  35.154811   7.959770   1.632184   4.333333  21.229524          6   
 657   34.916667   7.166667   1.750000   5.500000  20.500000          6   
 
       LOW_SAMPLE  
 1192       False  
 1050       False  
 1209      

In [14]:
REPORTS_DIR = PROJECT_ROOT / "data" / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

csv_out = REPORTS_DIR / "league_replacement_radar_2024_25.csv"
radar.to_csv(csv_out, index=False)

print("Saved:", csv_out)
print("Exists:", csv_out.exists())
print("Bytes:", csv_out.stat().st_size)


Saved: C:\Users\bo_co\ai-sports-competitive-intel\nba\data\reports\league_replacement_radar_2024_25.csv
Exists: True
Bytes: 184179


In [15]:
import pandas as pd
pd.read_csv(csv_out).shape



(1309, 11)

In [16]:
radar.info()
radar.head()


<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 1192 to 1305
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   STAR_OUT     1309 non-null   object 
 1   TEAM_CITY    1309 non-null   object 
 2   TEAM_NAME    1309 non-null   object 
 3   BENEFICIARY  1309 non-null   object 
 4   SCORE        1188 non-null   float64
 5   DELTA_PTS    1209 non-null   float64
 6   DELTA_AST    1209 non-null   float64
 7   DELTA_REB    1209 non-null   float64
 8   DELTA_MIN    1188 non-null   float64
 9   N_WITHOUT    1309 non-null   int64  
 10  LOW_SAMPLE   1309 non-null   bool   
dtypes: bool(1), float64(5), int64(1), object(4)
memory usage: 113.8+ KB


,STAR_OUT,TEAM_CITY,TEAM_NAME,BENEFICIARY,SCORE,DELTA_PTS,DELTA_AST,DELTA_REB,DELTA_MIN,N_WITHOUT,LOW_SAMPLE
1192,Trae Young,Atlanta,Hawks,Keaton Wallace,38.849074,10.344444,5.977778,3.522222,19.004630,6,False
1050,Paul George,Philadelphia,76ers,Chuma Okeke,38.500000,8.800000,2.200000,7.000000,20.500000,5,False
1209,Trey Murphy III,New Orleans,Pelicans,Keion Brooks Jr.,35.504747,9.972222,0.805556,3.416667,21.310303,12,False
1158,Shai Gilgeous-Alexander,Oklahoma City,Thunder,Dillon Jones,35.154811,7.959770,1.632184,4.333333,21.229524,6,False
657,Joel Embiid,Philadelphia,76ers,Chuma Okeke,34.916667,7.166667,1.750000,5.500000,20.500000,6,False


In [17]:
required_cols = [
    "STAR_OUT", "TEAM_CITY", "TEAM_NAME", "BENEFICIARY",
    "DELTA_PTS", "DELTA_MIN", "DELTA_AST", "DELTA_REB",
    "N_WITHOUT", "LOW_SAMPLE"
]

missing = [c for c in required_cols if c not in radar.columns]
missing


[]

In [18]:
radar[["DELTA_PTS", "DELTA_MIN", "DELTA_AST", "DELTA_REB"]].describe()


,DELTA_PTS,DELTA_MIN,DELTA_AST,DELTA_REB
count,1209.000000,1188.000000,1209.000000,1209.000000
mean,1.583388,2.714583,0.403231,0.530780
std,3.316784,5.212538,0.927546,1.320603
min,-14.314394,-22.914000,-3.252101,-3.978802
25%,-0.346154,-0.514464,-0.096154,-0.215716
50%,1.316502,2.193468,0.277126,0.454545
75%,3.375610,5.396980,0.809524,1.168831
max,16.592593,23.586271,5.977778,7.000000


In [19]:
radar[["DELTA_PTS", "DELTA_MIN"]].mean()


DELTA_PTS    1.583388
DELTA_MIN    2.714583
dtype: float64

In [20]:
radar["LOW_SAMPLE"].value_counts()


LOW_SAMPLE
False    1309
Name: count, dtype: int64

In [21]:
radar.groupby("LOW_SAMPLE")[["DELTA_PTS", "DELTA_MIN"]].mean()


,DELTA_PTS,DELTA_MIN
LOW_SAMPLE,,
False,1.583388,2.714583


In [22]:
radar.sort_values("DELTA_PTS", ascending=False).head(10)[
    ["STAR_OUT", "BENEFICIARY", "TEAM_NAME", "DELTA_PTS", "DELTA_MIN", "N_WITHOUT"]
]


,STAR_OUT,BENEFICIARY,TEAM_NAME,DELTA_PTS,DELTA_MIN,N_WITHOUT
31,Anthony Davis,Kyrie Irving,Mavericks,16.592593,-6.324314,54
1089,Quentin Grimes,Tyrese Maxey,76ers,16.256803,-0.073130,49
952,Miles Bridges,LaMelo Ball,Hornets,16.112957,6.636905,7
325,DeMar DeRozan,De'Aaron Fox,Kings,14.733333,2.841273,5
1153,Shai Gilgeous-Alexander,Aaron Wiggins,Thunder,14.473333,10.927732,6
1077,Quentin Grimes,Jared McCain,76ers,14.321429,NaN,28
1235,Tyrese Maxey,Jared McCain,76ers,14.297101,10.127143,6
918,Malik Monk,De'Aaron Fox,Kings,11.916667,2.402012,8
670,Joel Embiid,Lonnie Walker IV,76ers,11.761905,15.063684,21
496,Giannis Antetokounmpo,Damian Lillard,Bucks,11.422951,3.420500,5


In [23]:
radar["VALIDATED"] = ~radar["LOW_SAMPLE"]


In [24]:
validated_out = REPORTS_DIR / "league_replacement_radar_2024_25_validated.csv"
radar.to_csv(validated_out, index=False)

validated_out, validated_out.exists()


(WindowsPath('C:/Users/bo_co/ai-sports-competitive-intel/nba/data/reports/league_replacement_radar_2024_25_validated.csv'),
 True)